<a href="https://colab.research.google.com/github/JuhyupKim/HY_DIC/blob/main/124-Deeplearning_mnist05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Data import

In [ ]:
traindata_url = 'https://bitbucket.org/hyuk125/lg_dic/raw/889649d1bc273bf53967cb621a370205715b803b/dataset_day1/mnist_train.csv'
testdata_url = 'https://bitbucket.org/hyuk125/lg_dic/raw/889649d1bc273bf53967cb621a370205715b803b/dataset_day1/mnist_test.csv'
train_data = pd.read_csv(traindata_url)
test_data = pd.read_csv(testdata_url)

## 데이터 확인

In [ ]:
def plot_digit(data):
    image = data.reshape(28, 28) # 1d vector를 28*28 형태로 변경
    plt.imshow(image, cmap = matplotlib.cm.binary, 
               interpolation="nearest")
    plt.axis("off")

In [ ]:
train_data

In [ ]:
index = 600
plot_digit(train_data.values[index, 1:])
plt.show()
print('label: ', train_data.values[index, 0])

In [ ]:

def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap = matplotlib.cm.binary,
               interpolation="nearest")
    plt.axis("off")

In [ ]:
# 숫자 그림을 위한 추가 함수
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = matplotlib.cm.binary, **options)
    plt.axis("off")

In [ ]:

plt.figure(figsize=(9,9))
example_images = train_data.values[:60000:600, 1:]
plot_digits(example_images, images_per_row=10)
plt.show()

## Convert to 05 data

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

le.fit(train_data.label == 5)

train_data.label = le.transform(train_data.label == 5)
test_data.label = le.transform(test_data.label == 5)

## Deep learning - classification 모델

### Pytorch 모델에 입력하기 위한 데이터 변환

In [ ]:
train_data = torch.from_numpy(train_data.values).float()
test_data = torch.from_numpy(test_data.values).float()

In [ ]:
BATCH_SIZE = 15
epochs = 2
learning_rate = 0.001

In [ ]:

data_loader = torch.utils.data.DataLoader(train_data,
                            batch_size=BATCH_SIZE, 
                            shuffle=True, 
                            num_workers=0)

### Deep learning 모델 정의 

In [ ]:
class DNNModel(nn.Module):
    def __init__(self):
        super(DNNModel, self).__init__()
        self.layer1 = nn.Linear(28 *28, 300)
        self.layer2 = nn.Linear(300, 2)
        self.relu = nn.ReLU()
    
        
    def forward(self, x):
        
        layers = nn.Sequential(self.layer1, 
                               self.relu,
                               self.layer2, 
                               self.relu
                               ).to(device)
        out = layers(x)
        return out
    
model = DNNModel()
model

In [ ]:
class DNNModel(nn.Module):
    def __init__(self):
        super(DNNModel, self).__init__()
        self.layer1 = nn.Linear(28 *28, 300)
        self.layer2 = nn.Linear(300, 100)
        self.layer3 = nn.Linear(100, 2)
        self.relu = nn.ReLU()
    
        
    def forward(self, x):
        
        layers = nn.Sequential(self.layer1, 
                               self.relu,
                               self.layer2, 
                               self.relu,
                               self.layer3).to(device)
        out = layers(x)
        return out
    
model = DNNModel()
model

### 학습 시작

GPU로 넘겨야 하는것
- model의 layer
- cost(criterion)
- data

#### torch.nn.CrossEntropyLoss() 함수에 대해
원래 cross entropy는 

In [ ]:
from IPython.display import Image
image_url = 'https://bitbucket.org/hyuk125/lg_dic/raw/99785e9d01523e8bb6bf78d1220c1b020fa6c7c8/image_day1/cross_entropy.png'
Image(image_url)

pytorh.nn.CrossEntropyLoss() 는

In [ ]:
from IPython.display import Image
image_url = 'https://bitbucket.org/hyuk125/lg_dic/raw/99785e9d01523e8bb6bf78d1220c1b020fa6c7c8/image_day1/torch_cross_entropy.png'
Image(image_url)

softmax와 label의 elemental wise 곱을하면 label이 0인 확률들은 모두 없어지게 되어 아래 수식이 가능  
따라서 class에는 one-hot encoding 되지 않은 값이 들어가야 함  
x에는 soft max를 거치지 않은 벡터가 들어감(CrossEntropyLoss()에 softmax가 포함)

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
for epoch in range(epochs):
    running_cost = 0.0

    for step, (batch_data) in enumerate(data_loader):
        batch_x = batch_data[:, 1:].view(-1, 28*28).to(device)
        batch_y = batch_data[:, 0].to(device).long()
        
        optimizer.zero_grad()
        
        outputs = model(batch_x)
        cost = criterion(outputs, batch_y)

        cost.backward()
        optimizer.step()
        
        running_cost += cost.item()
        if step % 200 == 199:
            print('[%d, %5d] cost: %.3f' % (epoch + 1, step + 1, running_cost / 200))
            running_cost = 0.0
            

## 정확도 판단

### Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

In [ ]:
with torch.no_grad():
    X_test = test_data[:, 1:].view(-1, 28 * 28).float().to(device)
    y_test = test_data[:, 0].float()
    
    prediction = model(X_test).cpu()
    print(confusion_matrix(y_test, torch.argmax(prediction, 1)))
    print("==Precision==")
    print(precision_score(y_test, torch.argmax(prediction, 1), average=None))
    print(precision_score(y_test, torch.argmax(prediction, 1), average='weighted'))
    print("Recall")
    print(recall_score(y_test, torch.argmax(prediction, 1), average=None))
    print(recall_score(y_test, torch.argmax(prediction, 1), average='weighted'))